# Iteration 12: multiple reps (3) stage 2

## Plain English Summary


## Notebook imports

In [1]:
from IPython.display import Markdown, display

import numpy as np
import pandas as pd
import math

## LLM Prompt 

In [2]:
display(Markdown("../04_prompts_stage1/inputs/iteration_12.md"))

create a function called "multiple_replications". This should be standalone and separate from the CCUModel class. The function accepts an instance of Experiment and the number of replications (default=5) as parameters. The functions logic of each replication is as follows:

1. create a new instance of the model
2. call the reset_kpi function in the experiment 
3. run the model and store the results returned in a 
4. store the returned results of the replication for later use.

After all multiple replications have been completed concatenate all results into a single dataframe. The first column of the dataframe should be an integer representing the replication number. The function returns the results.


## Date and time of input to LLM

> 1115 07th June 2024 

## Sources

No sources reported

## Observations on the LLM Output



## LLM Output 1 stage 2:

Here's the modified code with the multiple_replications function added. This function is standalone and separate from the CCU class. It accepts an instance of Experiment and the number of replications (default=5) as parameters. The function runs multiple replications of the model, stores the results, and concatenates them into a single dataframe with the replication number as the first column.

In [47]:
def multiple_replications(experiment, num_replications=5):
    all_results = []

    for i in range(num_replications):
        env = simpy.Environment()
        ccu = CCU(env, experiment)
        ccu.reset_kpi()
        results = ccu.run()
        results['Replication'] = i + 1
        all_results.append(results)

    combined_results = pd.concat(all_results, ignore_index=True)
    return combined_results


## `CCUModel` and `Experiment` Additional prompt:

Modify the modified code as follows: Add patient_id_counter into the results dataframe as 'Patient Count'. Transpose the dataframe after creating it with an index starting from 0.

In [48]:
import simpy
import numpy as np
import pandas as pd
from scipy.stats import lognorm

class Experiment:
    def __init__(self, accident_emergency_arrival_rate=22.72, wards_arrival_rate=26.0, emergency_surgery_arrival_rate=37.0, other_hospitals_arrival_rate=47.2, xray_department_arrival_rate=575.0, elective_surgery_arrival_rate=17.91, elective_surgery_arrival_std_dev=3.16, accident_emergency_lognormal_mu=128.79, accident_emergency_lognormal_sigma=267.51, wards_lognormal_mu=177.89, wards_lognormal_sigma=276.54, emergency_surgery_lognormal_mu=140.15, emergency_surgery_lognormal_sigma=218.02, other_hospitals_lognormal_mu=212.86, other_hospitals_lognormal_sigma=457.67, xray_department_lognormal_mu=87.53, xray_department_lognormal_sigma=108.67, elective_surgery_treatment_rate=57.34, num_critical_care_beds=24, intensive_cleaning_duration=5, results_collection_period=12 * 30 * 24, warm_up_period=30 * 24, trace=False):
        self.accident_emergency_arrival_rate = accident_emergency_arrival_rate
        self.wards_arrival_rate = wards_arrival_rate
        self.emergency_surgery_arrival_rate = emergency_surgery_arrival_rate
        self.other_hospitals_arrival_rate = other_hospitals_arrival_rate
        self.xray_department_arrival_rate = xray_department_arrival_rate
        self.elective_surgery_arrival_rate = elective_surgery_arrival_rate
        self.elective_surgery_arrival_std_dev = elective_surgery_arrival_std_dev
        self.accident_emergency_lognormal_mu = accident_emergency_lognormal_mu
        self.accident_emergency_lognormal_sigma = accident_emergency_lognormal_sigma
        self.wards_lognormal_mu = wards_lognormal_mu
        self.wards_lognormal_sigma = wards_lognormal_sigma
        self.emergency_surgery_lognormal_mu = emergency_surgery_lognormal_mu
        self.emergency_surgery_lognormal_sigma = emergency_surgery_lognormal_sigma
        self.other_hospitals_lognormal_mu = other_hospitals_lognormal_mu
        self.other_hospitals_lognormal_sigma = other_hospitals_lognormal_sigma
        self.xray_department_lognormal_mu = xray_department_lognormal_mu
        self.xray_department_lognormal_sigma = xray_department_lognormal_sigma
        self.elective_surgery_treatment_rate = elective_surgery_treatment_rate
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.results_collection_period = results_collection_period
        self.warm_up_period = warm_up_period
        self.total_run_length = self.warm_up_period + self.results_collection_period
        self.trace = trace

class CCU:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_id_counter = 0
        self.cancelled_operations = 0
        self.warmup_end_time = self.experiment.warm_up_period
        self.total_unplanned_waiting_time = 0
        self.total_unplanned_admissions = 0
        self.total_treatment_time = 0
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)
        self.rng_elective_surgery = np.random.default_rng(seed=47)
        self.critical_care_beds = simpy.Resource(env, capacity=self.experiment.num_critical_care_beds)

    def lognormal_to_normal(self, mu, sigma):
        zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
        sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
        mu_norm = zeta
        return mu_norm, sigma_norm

    def warmup_complete(self):
        yield self.env.timeout(self.warmup_end_time)
        self.patient_id_counter = 0
        if self.experiment.trace:
            print("Warm-up complete")

    def reset_kpi(self):
        """Reset all performance measures to their original values"""
        self.cancelled_operations = 0
        self.total_unplanned_waiting_time = 0
        self.total_unplanned_admissions = 0
        self.total_treatment_time = 0
        self.patient_id_counter = 0  # Reset patient_id_counter

    def accident_emergency_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_accident_emergency.exponential(self.experiment.accident_emergency_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Accident and Emergency at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Accident and Emergency"))

    def wards_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_wards.exponential(self.experiment.wards_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from the Wards at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Wards"))

    def emergency_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_emergency_surgery.exponential(self.experiment.emergency_surgery_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Emergency Surgery at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Emergency Surgery"))

    def other_hospitals_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_other_hospitals.exponential(self.experiment.other_hospitals_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Other Hospitals at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Other Hospitals"))

    def xray_department_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_xray_department.exponential(self.experiment.xray_department_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from the X-Ray Department at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("X-Ray Department"))

    def elective_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_elective_surgery.normal(self.experiment.elective_surgery_arrival_rate, self.experiment.elective_surgery_arrival_std_dev))
            if self.experiment.trace:
                print(f"Elective Patient {self.patient_id_counter} arrived at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.elective_admissions_process())

    def unplanned_admissions_process(self, source):
        patient_id = self.patient_id_counter
        if self.experiment.trace:
            print(f"Patient {patient_id} from {source} requests a critical care bed at {self.env.now:.2f} hours")
        waiting_time = self.env.now
        with self.critical_care_beds.request() as req:
            yield req
            waiting_time = self.env.now - waiting_time
            if self.experiment.trace:
                print(f"Patient {patient_id} from {source} waited {waiting_time:.2f} hours")
                print(f"Patient {patient_id} from {source} admitted to a critical care bed at {self.env.now:.2f} hours")

            if source == "Accident and Emergency":
                mu, sigma = self.lognormal_to_normal(self.experiment.accident_emergency_lognormal_mu, self.experiment.accident_emergency_lognormal_sigma)
                length_of_stay = self.rng_accident_emergency.lognormal(mu, sigma)
            elif source == "Wards":
                mu, sigma = self.lognormal_to_normal(self.experiment.wards_lognormal_mu, self.experiment.wards_lognormal_sigma)
                length_of_stay = self.rng_wards.lognormal(mu, sigma)
            elif source == "Emergency Surgery":
                mu, sigma = self.lognormal_to_normal(self.experiment.emergency_surgery_lognormal_mu, self.experiment.emergency_surgery_lognormal_sigma)
                length_of_stay = self.rng_emergency_surgery.lognormal(mu, sigma)
            elif source == "Other Hospitals":
                mu, sigma = self.lognormal_to_normal(self.experiment.other_hospitals_lognormal_mu, self.experiment.other_hospitals_lognormal_sigma)
                length_of_stay = self.rng_other_hospitals.lognormal(mu, sigma)
            else:
                mu, sigma = self.lognormal_to_normal(self.experiment.xray_department_lognormal_mu, self.experiment.xray_department_lognormal_sigma)
                length_of_stay = self.rng_xray_department.lognormal(mu, sigma)

            yield self.env.timeout(length_of_stay)

            if self.env.now >= self.warmup_end_time:
                self.total_treatment_time += length_of_stay
                self.total_unplanned_waiting_time += waiting_time
                self.total_unplanned_admissions += 1

            if self.experiment.trace:
                print(f"Patient {patient_id} from {source} discharged from a critical care bed at {self.env.now:.2f} hours")

            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            if self.experiment.trace:
                print(f"Intensive cleaning completed for Patient {patient_id} from {source} at {self.env.now:.2f} hours")

    def elective_admissions_process(self):
        patient_id = self.patient_id_counter
        if self.critical_care_beds.count == self.critical_care_beds.capacity:
            if self.env.now >= self.warmup_end_time:
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds")
                self.cancelled_operations += 1
            else:
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds (warm-up period)")
        else:
            if self.experiment.trace:
                print(f"Elective Patient {patient_id} requests a critical care bed at {self.env.now:.2f} hours")
            with self.critical_care_beds.request() as req:
                yield req
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} admitted to a critical care bed at {self.env.now:.2f} hours")
                length_of_stay = self.rng_elective_surgery.exponential(self.experiment.elective_surgery_treatment_rate)
                yield self.env.timeout(length_of_stay)
                if self.env.now >= self.warmup_end_time:
                    self.total_treatment_time += length_of_stay
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} discharged from a critical care bed at {self.env.now:.2f} hours")
                yield self.env.timeout(self.experiment.intensive_cleaning_duration)
                if self.experiment.trace:
                    print(f"Intensive cleaning completed for Elective Patient {patient_id} at {self.env.now:.2f} hours")

    def run(self):
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())
        elective_surgery_process = self.env.process(self.elective_surgery_arrivals())
        warmup_complete_process = self.env.process(self.warmup_complete())
        self.env.run(until=self.experiment.total_run_length)

        # Calculate bed utilization as a proportion
        bed_utilization = self.total_treatment_time / (self.experiment.num_critical_care_beds * self.experiment.results_collection_period)

        # Calculate bed occupancy
        bed_occupancy = bed_utilization * self.experiment.num_critical_care_beds

        performance_measures = {
            "Total Cancelled Elective Operations": self.cancelled_operations,
            "Mean Unplanned Admission Waiting Time (hours)": self.total_unplanned_waiting_time / self.total_unplanned_admissions if self.total_unplanned_admissions > 0 else 0,
            "Bed Utilization": bed_utilization,
            "Bed Occupancy": bed_occupancy,
            "Patient Count": self.patient_id_counter
        }

        results_df = pd.DataFrame.from_dict(performance_measures, orient='index').T
        return results_df

# Run the simulation
experiment = Experiment(trace=False)  # Set trace=True to enable event printing
env = simpy.Environment()
ccu = CCU(env, experiment)
ccu.run()


,Total Cancelled Elective Operations,Mean Unplanned Admission Waiting Time (hours),Bed Utilization,Bed Occupancy,Patient Count
0,324.0,24.652835,0.884274,21.222565,1642.0


In [49]:
# Example usage (written by LLM)
experiment = Experiment(trace=False)
results = multiple_replications(experiment, num_replications=5)
results


,Total Cancelled Elective Operations,Mean Unplanned Admission Waiting Time (hours),Bed Utilization,Bed Occupancy,Patient Count,Replication
0,324.0,24.652835,0.884274,21.222565,1642.0,1
1,324.0,24.652835,0.884274,21.222565,1642.0,2
2,324.0,24.652835,0.884274,21.222565,1642.0,3
3,324.0,24.652835,0.884274,21.222565,1642.0,4
4,324.0,24.652835,0.884274,21.222565,1642.0,5


## Replications test

The LLM introduced random number generator (RNG) seeds, and each replication has identical results. In the code below, the RNG has been turned off and the use of RNG will be checked in a future iteration. 

In [68]:
import simpy
import numpy as np
import pandas as pd
from scipy.stats import lognorm

class Experiment:
    def __init__(self, accident_emergency_arrival_rate=22.72, wards_arrival_rate=26.0, emergency_surgery_arrival_rate=37.0, other_hospitals_arrival_rate=47.2, xray_department_arrival_rate=575.0, elective_surgery_arrival_rate=17.91, elective_surgery_arrival_std_dev=3.16, accident_emergency_lognormal_mu=128.79, accident_emergency_lognormal_sigma=267.51, wards_lognormal_mu=177.89, wards_lognormal_sigma=276.54, emergency_surgery_lognormal_mu=140.15, emergency_surgery_lognormal_sigma=218.02, other_hospitals_lognormal_mu=212.86, other_hospitals_lognormal_sigma=457.67, xray_department_lognormal_mu=87.53, xray_department_lognormal_sigma=108.67, elective_surgery_treatment_rate=57.34, num_critical_care_beds=24, intensive_cleaning_duration=5, results_collection_period=12 * 30 * 24, warm_up_period=30 * 24, trace=False):
        self.accident_emergency_arrival_rate = accident_emergency_arrival_rate
        self.wards_arrival_rate = wards_arrival_rate
        self.emergency_surgery_arrival_rate = emergency_surgery_arrival_rate
        self.other_hospitals_arrival_rate = other_hospitals_arrival_rate
        self.xray_department_arrival_rate = xray_department_arrival_rate
        self.elective_surgery_arrival_rate = elective_surgery_arrival_rate
        self.elective_surgery_arrival_std_dev = elective_surgery_arrival_std_dev
        self.accident_emergency_lognormal_mu = accident_emergency_lognormal_mu
        self.accident_emergency_lognormal_sigma = accident_emergency_lognormal_sigma
        self.wards_lognormal_mu = wards_lognormal_mu
        self.wards_lognormal_sigma = wards_lognormal_sigma
        self.emergency_surgery_lognormal_mu = emergency_surgery_lognormal_mu
        self.emergency_surgery_lognormal_sigma = emergency_surgery_lognormal_sigma
        self.other_hospitals_lognormal_mu = other_hospitals_lognormal_mu
        self.other_hospitals_lognormal_sigma = other_hospitals_lognormal_sigma
        self.xray_department_lognormal_mu = xray_department_lognormal_mu
        self.xray_department_lognormal_sigma = xray_department_lognormal_sigma
        self.elective_surgery_treatment_rate = elective_surgery_treatment_rate
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.results_collection_period = results_collection_period
        self.warm_up_period = warm_up_period
        self.total_run_length = self.warm_up_period + self.results_collection_period
        self.trace = trace

class CCU:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_id_counter = 0
        self.cancelled_operations = 0
        self.warmup_end_time = self.experiment.warm_up_period
        self.total_unplanned_waiting_time = 0
        self.total_unplanned_admissions = 0
        self.total_treatment_time = 0
        self.rng_accident_emergency = np.random.default_rng()
        self.rng_wards = np.random.default_rng()
        self.rng_emergency_surgery = np.random.default_rng()
        self.rng_other_hospitals = np.random.default_rng()
        self.rng_xray_department = np.random.default_rng()
        self.rng_elective_surgery = np.random.default_rng()
        self.critical_care_beds = simpy.Resource(env, capacity=self.experiment.num_critical_care_beds)

    def lognormal_to_normal(self, mu, sigma):
        zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
        sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
        mu_norm = zeta
        return mu_norm, sigma_norm

    def warmup_complete(self):
        yield self.env.timeout(self.warmup_end_time)
        self.patient_id_counter = 0
        if self.experiment.trace:
            print("Warm-up complete")

    def reset_kpi(self):
        """Reset all performance measures to their original values"""
        self.cancelled_operations = 0
        self.total_unplanned_waiting_time = 0
        self.total_unplanned_admissions = 0
        self.total_treatment_time = 0
        self.patient_id_counter = 0  # Reset patient_id_counter

    def accident_emergency_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_accident_emergency.exponential(self.experiment.accident_emergency_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Accident and Emergency at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Accident and Emergency"))

    def wards_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_wards.exponential(self.experiment.wards_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from the Wards at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Wards"))

    def emergency_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_emergency_surgery.exponential(self.experiment.emergency_surgery_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Emergency Surgery at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Emergency Surgery"))

    def other_hospitals_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_other_hospitals.exponential(self.experiment.other_hospitals_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Other Hospitals at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Other Hospitals"))

    def xray_department_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_xray_department.exponential(self.experiment.xray_department_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from the X-Ray Department at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("X-Ray Department"))

    def elective_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_elective_surgery.normal(self.experiment.elective_surgery_arrival_rate, self.experiment.elective_surgery_arrival_std_dev))
            if self.experiment.trace:
                print(f"Elective Patient {self.patient_id_counter} arrived at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.elective_admissions_process())

    def unplanned_admissions_process(self, source):
        patient_id = self.patient_id_counter
        if self.experiment.trace:
            print(f"Patient {patient_id} from {source} requests a critical care bed at {self.env.now:.2f} hours")
        waiting_time = self.env.now
        with self.critical_care_beds.request() as req:
            yield req
            waiting_time = self.env.now - waiting_time
            if self.experiment.trace:
                print(f"Patient {patient_id} from {source} waited {waiting_time:.2f} hours")
                print(f"Patient {patient_id} from {source} admitted to a critical care bed at {self.env.now:.2f} hours")

            if source == "Accident and Emergency":
                mu, sigma = self.lognormal_to_normal(self.experiment.accident_emergency_lognormal_mu, self.experiment.accident_emergency_lognormal_sigma)
                length_of_stay = self.rng_accident_emergency.lognormal(mu, sigma)
            elif source == "Wards":
                mu, sigma = self.lognormal_to_normal(self.experiment.wards_lognormal_mu, self.experiment.wards_lognormal_sigma)
                length_of_stay = self.rng_wards.lognormal(mu, sigma)
            elif source == "Emergency Surgery":
                mu, sigma = self.lognormal_to_normal(self.experiment.emergency_surgery_lognormal_mu, self.experiment.emergency_surgery_lognormal_sigma)
                length_of_stay = self.rng_emergency_surgery.lognormal(mu, sigma)
            elif source == "Other Hospitals":
                mu, sigma = self.lognormal_to_normal(self.experiment.other_hospitals_lognormal_mu, self.experiment.other_hospitals_lognormal_sigma)
                length_of_stay = self.rng_other_hospitals.lognormal(mu, sigma)
            else:
                mu, sigma = self.lognormal_to_normal(self.experiment.xray_department_lognormal_mu, self.experiment.xray_department_lognormal_sigma)
                length_of_stay = self.rng_xray_department.lognormal(mu, sigma)

            yield self.env.timeout(length_of_stay)

            if self.env.now >= self.warmup_end_time:
                self.total_treatment_time += length_of_stay
                self.total_unplanned_waiting_time += waiting_time
                self.total_unplanned_admissions += 1

            if self.experiment.trace:
                print(f"Patient {patient_id} from {source} discharged from a critical care bed at {self.env.now:.2f} hours")

            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            if self.experiment.trace:
                print(f"Intensive cleaning completed for Patient {patient_id} from {source} at {self.env.now:.2f} hours")

    def elective_admissions_process(self):
        patient_id = self.patient_id_counter
        if self.critical_care_beds.count == self.critical_care_beds.capacity:
            if self.env.now >= self.warmup_end_time:
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds")
                self.cancelled_operations += 1
            else:
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds (warm-up period)")
        else:
            if self.experiment.trace:
                print(f"Elective Patient {patient_id} requests a critical care bed at {self.env.now:.2f} hours")
            with self.critical_care_beds.request() as req:
                yield req
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} admitted to a critical care bed at {self.env.now:.2f} hours")
                length_of_stay = self.rng_elective_surgery.exponential(self.experiment.elective_surgery_treatment_rate)
                yield self.env.timeout(length_of_stay)
                if self.env.now >= self.warmup_end_time:
                    self.total_treatment_time += length_of_stay
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} discharged from a critical care bed at {self.env.now:.2f} hours")
                yield self.env.timeout(self.experiment.intensive_cleaning_duration)
                if self.experiment.trace:
                    print(f"Intensive cleaning completed for Elective Patient {patient_id} at {self.env.now:.2f} hours")

    def run(self):
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())
        elective_surgery_process = self.env.process(self.elective_surgery_arrivals())
        warmup_complete_process = self.env.process(self.warmup_complete())
        self.env.run(until=self.experiment.total_run_length)

        # Calculate bed utilization as a proportion
        bed_utilization = self.total_treatment_time / (self.experiment.num_critical_care_beds * self.experiment.results_collection_period)

        # Calculate bed occupancy
        bed_occupancy = bed_utilization * self.experiment.num_critical_care_beds

        performance_measures = {
            "Total Cancelled Elective Operations": self.cancelled_operations,
            "Mean Unplanned Admission Waiting Time (hours)": self.total_unplanned_waiting_time / self.total_unplanned_admissions if self.total_unplanned_admissions > 0 else 0,
            "Bed Utilization": bed_utilization,
            "Bed Occupancy": bed_occupancy,
            "Patient Count": self.patient_id_counter
        }

        results_df = pd.DataFrame.from_dict(performance_measures, orient='index').T
        return results_df



In [69]:
# Run the simulation
experiment = Experiment(trace=False)  # Set trace=True to enable event printing
env = simpy.Environment()
ccu = CCU(env, experiment)
ccu.run()

,Total Cancelled Elective Operations,Mean Unplanned Admission Waiting Time (hours),Bed Utilization,Bed Occupancy,Patient Count
0,300.0,24.685258,0.911154,21.867691,1589.0


In [70]:
# Example usage (written by LLM)
experiment = Experiment(trace=False)
results = multiple_replications(experiment, num_replications=5)
results


,Total Cancelled Elective Operations,Mean Unplanned Admission Waiting Time (hours),Bed Utilization,Bed Occupancy,Patient Count,Replication
0,328.0,30.490467,0.923285,22.158834,1627.0,1
1,328.0,113.127922,0.926000,22.224012,1644.0,2
2,455.0,196.263368,0.947494,22.739861,1675.0,3
3,252.0,11.444558,0.892683,21.424391,1610.0,4
4,274.0,18.084559,0.892092,21.410219,1656.0,5


## Testing

Here we do a number of regression tests to check that previous functionality is not affected by the `multiple_replications` function. 

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Elective."
* **Extreme value test 3**: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result **(PASS)**: queues form after first arrival. 
* **Extreme value test 4**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result **(PASS)**: after 24 arrivals queues form and no patients are admitted.
* **Vary warm up period**: Vary the warm-up period while holding the results collection period constant.
    * Expected result: Run length is equal to the `results_collection_period`
    * Actual result **(PASS)**: Run length is equal to the `results_collection_period`

In [71]:
# The extreme value
M = 10_000_000

### Test 1: Extreme value test 1

PASS

In [87]:
def run_test(experiment):
    # Create a SimPy environment
    env = simpy.Environment()
    ccu_model = CCU(env, experiment)
    results = ccu_model.run()
    return ccu_model, results

In [88]:
def extreme_value_test_1(extreme_value=M*5):
    '''
    Extreme value test 1: 
    
    Wards, Em Surgery, other hospitals, x-ray, electives have their inter-arrival time
    set to $M$ a very large number
    Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    '''
    experiment = Experiment(wards_arrival_rate = extreme_value,
                            emergency_surgery_arrival_rate = extreme_value, other_hospitals_arrival_rate = extreme_value,
                            xray_department_arrival_rate = extreme_value, elective_surgery_arrival_rate = extreme_value,
                            trace=False)
    return run_test(experiment)

In [89]:
model,results = extreme_value_test_1()
results

,Total Cancelled Elective Operations,Mean Unplanned Admission Waiting Time (hours),Bed Utilization,Bed Occupancy,Patient Count
0,0.0,0.0,0.269632,6.471172,408.0


### Test 2: Extreme value test 2

PASS

In [90]:
def extreme_value_test_2(extreme_value=M*5):
    '''
    Extreme value test 2: 
    
    All unplanned admissions have their inter-arrival time
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    experiment = Experiment(accident_emergency_arrival_rate = extreme_value, wards_arrival_rate = extreme_value,
                            emergency_surgery_arrival_rate = extreme_value, other_hospitals_arrival_rate = extreme_value,
                            xray_department_arrival_rate = extreme_value, trace=True)
    return run_test(experiment)

In [91]:
try:
    model, results = extreme_value_test_2()
except ZeroDivisionError:
    print("caught zero division error due to extreme value test")

Elective Patient 0 arrived at 17.25 hours
Elective Patient 1 requests a critical care bed at 17.25 hours
Elective Patient 1 admitted to a critical care bed at 17.25 hours
Elective Patient 1 arrived at 35.95 hours
Elective Patient 2 requests a critical care bed at 35.95 hours
Elective Patient 2 admitted to a critical care bed at 35.95 hours
Elective Patient 2 arrived at 51.91 hours
Elective Patient 3 requests a critical care bed at 51.91 hours
Elective Patient 3 admitted to a critical care bed at 51.91 hours
Elective Patient 3 arrived at 68.96 hours
Elective Patient 4 requests a critical care bed at 68.96 hours
Elective Patient 4 admitted to a critical care bed at 68.96 hours
Elective Patient 2 discharged from a critical care bed at 71.47 hours
Elective Patient 4 discharged from a critical care bed at 74.96 hours
Intensive cleaning completed for Elective Patient 2 at 76.47 hours
Intensive cleaning completed for Elective Patient 4 at 79.96 hours
Elective Patient 4 arrived at 84.43 hours


### Test 3: Extreme value test 3

PASS.

In [102]:
def extreme_value_test_3(critical_care_beds=1):
    '''
    Extreme value test 3: 
    
    Critical care beds set to 1 or parameter
    
    Expected result: when critical_care_beds=1 queues form after first arrival.
    Cancellations also begin after 1st arrival.

    Note: reported utilisation will not quite be 100% because of the patient still
    in service.
    
    '''
    experiment = Experiment(num_critical_care_beds=critical_care_beds)
    model, results = run_test(experiment)
    print(f'{model.total_treatment_time=}')
    print(f'{experiment.results_collection_period=}')
    return model, results

In [103]:
model, results = extreme_value_test_3()
results

model.total_treatment_time=8305.009410668828
experiment.results_collection_period=8640


,Total Cancelled Elective Operations,Mean Unplanned Admission Waiting Time (hours),Bed Utilization,Bed Occupancy,Patient Count
0,479.0,4236.048211,0.961228,0.961228,1616.0


### Test 5: Extreme value test 4

PASS

In [104]:
def extreme_value_test_4(extreme_value=M):
    '''
    Extreme value test 4: 
    
    means of stay_distributions set to $M$ a very large number.
    Expected result: after 24 arrivals queues form and no patients are admitted.

    Note: as patients do not leave beds, wait time, utilisation, and occupancy are not calculated.
    '''
    experiment = Experiment(accident_emergency_lognormal_mu=M, 
                 wards_lognormal_mu=M, 
                 emergency_surgery_lognormal_mu=M, 
                 other_hospitals_lognormal_mu=M, 
                 xray_department_lognormal_mu=M, 
                 elective_surgery_treatment_rate=M,
                 trace=True)
    return run_test(experiment)

In [105]:
try:
    extreme_value_test_4()
except ZeroDivisionError:
    print("caught expected zero division error due to extreme value test")

Patient 0 arrived from Emergency Surgery at 1.18 hours
Patient 1 from Emergency Surgery requests a critical care bed at 1.18 hours
Patient 1 from Emergency Surgery waited 0.00 hours
Patient 1 from Emergency Surgery admitted to a critical care bed at 1.18 hours
Elective Patient 1 arrived at 15.20 hours
Elective Patient 2 requests a critical care bed at 15.20 hours
Elective Patient 2 admitted to a critical care bed at 15.20 hours
Patient 2 arrived from the Wards at 24.33 hours
Patient 3 from Wards requests a critical care bed at 24.33 hours
Patient 3 from Wards waited 0.00 hours
Patient 3 from Wards admitted to a critical care bed at 24.33 hours
Patient 3 arrived from Accident and Emergency at 31.13 hours
Patient 4 from Accident and Emergency requests a critical care bed at 31.13 hours
Patient 4 from Accident and Emergency waited 0.00 hours
Patient 4 from Accident and Emergency admitted to a critical care bed at 31.13 hours
Patient 4 arrived from Accident and Emergency at 31.90 hours
Pat

### Test 6: Vary Warm-up parameter

PASS

In [106]:
def test_warmup_period(new_warmup_period):
    '''
    Vary the warm-up period while holding the 
    results collection period constant.

    Expected result Run length should be no longer than results_collection_period
    '''
    experiment = Experiment(warm_up_period=new_warmup_period)
    model, results = run_test(experiment)
    return model.env.now, experiment.results_collection_period

In [107]:
test_warmup_period(0)

(8640, 8640)